In [1]:
import itertools
import pdb
import os
from helper import *

In [2]:
def findsubsets(s, n): 
    #return list(itertools.combinations(s, n)) 
    return list(map(set, itertools.combinations(s, n))) 
s = {1, 2, 3} 
n = 2
  
print(findsubsets(s, n))

[{1, 2}, {1, 3}, {2, 3}]


In [3]:
def mes(apelido_fundo,anomes,tipo):
    if tipo == 'acoes':
        aux = pd.read_csv('excel_fundos\\'+apelido_fundo+'\\'+str(anomes)+'.xls',index_col = 0,sep='\t',decimal= '.')
        lista_acoes = ['Obrigações por ações e outros TVM recebidos em empréstimo',
                         'Ações e outros TVM cedidos em empréstimo',
                         'Ações',
                         'Valores a pagar']
        
        aux.pf_quant = aux.pf_quant.apply(abs)*aux.ativo1.apply(lambda x: -1 if 'VALORES A PAGAR' in x.upper() or 'OBRIGAÇÕES' in x.upper() else 1)
        aux.pf_valor_mercado = aux.pf_valor_mercado.apply(abs)*aux.ativo1.apply(lambda x: -1 if 'VALORES A PAGAR' in x.upper() or 'OBRIGAÇÕES' in x.upper() else 1)
        #aux.pf_quant = aux.pf_quant*abs(aux.porcent_partrimonio_liquido)/aux.porcent_partrimonio_liquido
        #aux.pf_valor_mercado = aux.pf_valor_mercado*abs(aux.porcent_partrimonio_liquido)/aux.porcent_partrimonio_liquido
        return aux[aux.ativo1.isin(lista_acoes)]

In [4]:
def precos(anomes,ativo):
    precos = pd.read_csv(str(anomes)[:-2]+'.txt',index_col=False,sep=';',header=0,chunksize=10000).read()
    precos = precos[(precos.CODNEG == ativo)&(precos.TPMERC == 10.0)&(precos.DATA > 100*int(anomes))&(precos.DATA < 100+100*int(anomes))]
    return precos

In [5]:
def teste(carteira_0,carteira_1,ativo,anomes):
    if anomes[4:6] == '01':
        fechamento_0 = precos(str(int(anomes)-101+12),ativo).sort_values(by='DATA',ascending=False).iloc[0,10]
    else:
        fechamento_0 = precos(str(int(anomes)-1),ativo).sort_values(by='DATA',ascending=False).iloc[0,10]
    quantidade_referencia =100*( carteira_0[carteira_0.ativo3 == ativo].pf_quant.sum() - carteira_1[~carteira_1.fechado][carteira_1.ativo3 == ativo].pf_quant.sum() )
    auxiliar = carteira_1[carteira_1.fechado][carteira_1.candidatos.str.contains(ativo)][['ativo1','aserestudado','vendas_montante','aquis_montante','pf_valor_mercado','cand_pf_quant']].fillna(0).copy()
    
    preco = precos(anomes,ativo)
    maximo = max(preco.PREMAX)
    minimo = min(preco.PREMIN)
    fechamento_1 = preco.sort_values(by='DATA',ascending=False).iloc[0,10]
    
    #auxiliar['MAX'] = 100*auxiliar.cand_pf_quant +(auxiliar.vendas_montante+99)/minimo- auxiliar.aquis_montante/maximo
    auxiliar['MAX'] = (auxiliar.pf_valor_mercado+99)/fechamento_1 +(auxiliar.vendas_montante+99)/minimo - auxiliar.aquis_montante/maximo
    #auxiliar['MIN'] = 100*auxiliar.cand_pf_quant  +auxiliar.vendas_montante/maximo - (auxiliar.aquis_montante+99)/minimo 
    auxiliar['MIN'] = auxiliar.pf_valor_mercado/fechamento_1 +auxiliar.vendas_montante/maximo - (auxiliar.aquis_montante+99)/minimo 
    lista = []
    #print(fechamento_0,minimo,maximo,fechamento_1,quantidade_referencia,"\n",auxiliar)
    #pdb.set_trace()
    for k in range(1,len(auxiliar.index)+1):
        for j in findsubsets(auxiliar.index, k):
            if round(auxiliar[auxiliar.index.isin(j)].MAX.sum()) >= quantidade_referencia >= round(auxiliar[auxiliar.index.isin(j)].MIN.sum()):
                if len(auxiliar[auxiliar.index.isin(j)].ativo1) == len(set(auxiliar[auxiliar.index.isin(j)].ativo1)):
                    lista.append(j)
    
    if len(lista) == 1:
        print('-------------------'+ativo+'-------------------')
        
    else:
        print(ativo,quantidade_referencia,"\n",auxiliar,"\n",lista)
    carteira_1 = completa(ativo,lista[0],carteira_1)
    return carteira_1

In [6]:
def teste_fraco(carteira_0,carteira_1,ativo,anomes):
    if anomes[4:6] == '01':
        fechamento_0 = precos(str(int(anomes)-101+12),ativo).sort_values(by='DATA',ascending=False).iloc[0,10]
    else:
        fechamento_0 = precos(str(int(anomes)-1),ativo).sort_values(by='DATA',ascending=False).iloc[0,10]
    quantidade_referencia =100*( carteira_0[carteira_0.ativo3 == ativo].pf_quant.sum() - carteira_1[~carteira_1.fechado][carteira_1.ativo3 == ativo].pf_quant.sum() )
    auxiliar = carteira_1[carteira_1.fechado][carteira_1.candidatos.str.contains(ativo)][['ativo1','aserestudado','vendas_montante','aquis_montante','pf_valor_mercado','cand_pf_quant']].fillna(0).copy()
    
    preco = precos(anomes,ativo)
    maximo = max(preco.PREMAX)
    minimo = min(preco.PREMIN)
    fechamento_1 = preco.sort_values(by='DATA',ascending=False).iloc[0,10]
    
    #auxiliar['MAX'] = 100*auxiliar.cand_pf_quant +(auxiliar.vendas_montante+99)/minimo- auxiliar.aquis_montante/maximo
    auxiliar['MAX'] = (auxiliar.pf_valor_mercado+99)/fechamento_1 +(auxiliar.vendas_montante+99)/minimo - auxiliar.aquis_montante/maximo
    #auxiliar['MIN'] = 100*auxiliar.cand_pf_quant  +auxiliar.vendas_montante/maximo - (auxiliar.aquis_montante+99)/minimo 
    auxiliar['MIN'] = auxiliar.pf_valor_mercado/fechamento_1 +auxiliar.vendas_montante/maximo - (auxiliar.aquis_montante+99)/minimo 
    lista = []
    #print(fechamento_0,minimo,maximo,fechamento_1,quantidade_referencia,"\n",auxiliar)
    #pdb.set_trace()
    for k in range(1,len(auxiliar.index)+1):
        for j in findsubsets(auxiliar.index, k):
            if round(auxiliar[auxiliar.index.isin(j)].MAX.sum()) >= quantidade_referencia >= round(auxiliar[auxiliar.index.isin(j)].MIN.sum()):
                if len(auxiliar[auxiliar.index.isin(j)].ativo1) == len(set(auxiliar[auxiliar.index.isin(j)].ativo1)):
                    lista.append(j)
    
    if len(lista) == 1:
        print('-------------------'+ativo+'-------------------')
        carteira_1 = completa(ativo,lista[0],carteira_1)
    else:
        print(ativo,quantidade_referencia,"\n",auxiliar,"\n",lista)
    return carteira_1

In [7]:
def filtro_alfabetico(carteira,alfabetico,aserestudado_com_varios_ativo3=[]):
    if type(aserestudado_com_varios_ativo3) != list:
        aserestudado_com_varios_ativo3 = [aserestudado_com_varios_ativo3]
    teste = carteira.copy()
    for aserestudado,ativo3,grupo in teste[~teste.ativo3.isnull()][~teste.aserestudado.isin(aserestudado_com_varios_ativo3)][['aserestudado','ativo3','grupo']].values:
    #for aserestudado,ativo3,grupo in teste[['aserestudado','ativo3','grupo']].values:
        if alfabetico:
            dropar = teste[(teste.grupo==grupo)&(teste.aserestudado>aserestudado)&(teste.candidatos<=ativo3)].index
            teste.drop(list(dropar), inplace=True)
            dropar = teste[(teste.grupo==grupo)&(teste.aserestudado<aserestudado)&(teste.candidatos>=ativo3)].index
            teste.drop(list(dropar), inplace=True)
        else:
            dropar = teste[(teste.grupo==grupo)&(teste.aserestudado>aserestudado)&(teste.candidatos>=ativo3)].index
            teste.drop(list(dropar), inplace=True)
            dropar = teste[(teste.grupo==grupo)&(teste.aserestudado<aserestudado)&(teste.candidatos<=ativo3)].index
            teste.drop(list(dropar), inplace=True)
    return teste
    

In [8]:
def filtro_unico_index(carteira,aserestudado_sem_nenhum_ativo3=[]):
    carteira = carteira.copy()
    if type(aserestudado_sem_nenhum_ativo3) != list:
        aserestudado_sem_nenhum_ativo3 = [aserestudado_sem_nenhum_ativo3]
    CONFUSAO = carteira[~carteira.aserestudado.isin(aserestudado_sem_nenhum_ativo3)].groupby(by='aserestudado').count()[['ativo3']]
    if (CONFUSAO.ativo3==1).all():
        print('trabalho concluido'.upper())
        return carteira[(carteira.ativo3.notnull())|(carteira.aserestudado.isin(aserestudado_sem_nenhum_ativo3))].sort_values(['aserestudado','candidatos'])
    else:
        
        print(CONFUSAO[CONFUSAO.ativo3>1])
        print(CONFUSAO[CONFUSAO.ativo3==0])
        return carteira.sort_values(['aserestudado','candidatos'])

In [9]:
def separa_blocos(carteira_pandas):
    
    
    
    teste = carteira_pandas.sort_values(by='aserestudado').copy()


    assert len(teste[teste.ativo1.str.upper() == 'disponibilidades'.upper()].index )==1
    assert len(teste[teste.ativo1.str.upper() == 'Valores a receber'.upper()].index )==1



    alfabetico = teste[teste.ativo1.str.upper() == 'disponibilidades'.upper()].aserestudado.values[0] < teste[teste.ativo1.str.upper() == 'Valores a receber'.upper()].aserestudado.values[0] 

    if not alfabetico:
        teste =teste.sort_values(by=['candidatos'],ascending=False).sort_values(by=['aserestudado'])
    
    
    assert len(teste[teste.ativo1.str.upper() == 'Obrigações por ações e outros TVM recebidos em empréstimo'.upper()].index )==0
    assert len(teste[teste.ativo1.str.upper() == 'Outros valores mobiliÃ¡rios registrados na CVM objeto de oferta pública'.upper()].index )==0
    assert len(teste[teste.ativo1.str.upper() == 'Valores a pagar'.upper()].index )<=1

    disponibilidade = teste[teste.ativo1.str.upper() == 'disponibilidades'.upper()].aserestudado.values[0]
    valor_receber =teste[teste.ativo1.str.upper() == 'Valores a receber'.upper()].aserestudado.values[0]


    teste['grupo'] = 0
    
    if not alfabetico:
        print('ISSO ESTA ERRADO, QUANDO É NÃO ALFABETICO (olhar squadra 202004 o aserestudado 58331405 q vc vai entender)')
        print('Tento by passar usando a localização de algum Investimento no Exterior')
        #return

        investimento_exterior_aserestudado =teste[teste.ativo1.str.upper() == 'Investimento no Exterior'.upper()].aserestudado.values[0]



        #Aqui considero que se houver mais de um valor a pagar, ele vaão estar separados
        lista = ['Obrigações por ações e outros TVM recebidos em empréstimo',
                            'Ações',
                            'Certificado ou recibo de depósito de valores mobiliários',
                            'Valores a pagar']
        grupo_1 = teste[teste.aserestudado<valor_receber][teste.ativo1.isin(lista)].index
        teste.loc[grupo_1,'grupo']= 1

        lista = ['Ações e outros TVM cedidos em empréstimo']
        grupo_1 = teste[teste.aserestudado<valor_receber][teste.ativo1.isin(lista)].index
        teste.loc[grupo_1,'grupo']= 2





        #Aqui considero que se houver mais de um valor a pagar, ele vaão estar separados
        lista = ['Obrigações por ações e outros TVM recebidos em empréstimo',
                            'Outros valores mobiliÃ¡rios registrados na CVM objeto de oferta pública',
                            'Ações',
                            'Certificado ou recibo de depósito de valores mobiliários',
                            'Valores a pagar',
                            'Brazilian Depository Receipt - BDR']
        grupo_1 = teste[teste.aserestudado>investimento_exterior_aserestudado][teste.aserestudado<valor_receber][teste.ativo1.isin(lista)].index
        teste.loc[grupo_1,'grupo']= 3

        lista = ['Ações e outros TVM cedidos em empréstimo']
        grupo_1 = teste[teste.aserestudado>investimento_exterior_aserestudado][teste.aserestudado<valor_receber][teste.ativo1.isin(lista)].index
        teste.loc[grupo_1,'grupo']= 4

    else:
        #Aqui considero que se houver mais de um valor a pagar, ele vaão estar separados
        lista = ['Obrigações por ações e outros TVM recebidos em empréstimo',
                            'Ações',
                            'Certificado ou recibo de depósito de valores mobiliários',
                            'Valores a pagar']
        grupo_1 = teste[teste.aserestudado<disponibilidade][teste.ativo1.isin(lista)].index
        teste.loc[grupo_1,'grupo']= 1

        lista = ['Ações e outros TVM cedidos em empréstimo']
        grupo_1 = teste[teste.aserestudado<disponibilidade][teste.ativo1.isin(lista)].index
        teste.loc[grupo_1,'grupo']= 2





        #Aqui considero que se houver mais de um valor a pagar, ele vaão estar separados
        lista = ['Obrigações por ações e outros TVM recebidos em empréstimo',
                            'Outros valores mobiliÃ¡rios registrados na CVM objeto de oferta pública',
                            'Ações',
                            'Certificado ou recibo de depósito de valores mobiliários',
                            'Valores a pagar',
                            'Brazilian Depository Receipt - BDR']
        grupo_1 = teste[teste.aserestudado>valor_receber][teste.ativo1.isin(lista)].index
        teste.loc[grupo_1,'grupo']= 3

        lista = ['Ações e outros TVM cedidos em empréstimo']
        grupo_1 = teste[teste.aserestudado>valor_receber][teste.ativo1.isin(lista)].index
        teste.loc[grupo_1,'grupo']= 4





    for t in set(teste.grupo) :
        if t != 0 and teste[teste.grupo==t].ativo3.apply(pd.isna).all() :
            print('grupo',t,'todos os ativos1 são nulos')

    return teste,alfabetico

In [10]:
anomes = 202003
apelido_fundo = 'CONST'.upper()


%time carteira_pandas,fundo,fundo_cnpj,adm,adm_cnpj,pl,hora_recebimento = parse_html_tudo_int(apelido_fundo,anomes,candidatos=True)

Wall time: 892 ms


In [11]:
carteira_pandas,alfabetico = separa_blocos(carteira_pandas)
carteira_pandas[carteira_pandas.grupo == 1]
aux = carteira_pandas.copy()

ISSO ESTA ERRADO, QUANDO É NÃO ALFABETICO (olhar squadra 202004 o aserestudado 58331405 q vc vai entender)
Tento by passar usando a localização de algum Investimento no Exterior


In [12]:
anomes = 202004
apelido_fundo = 'CONST'.upper()


%time carteira_pandas,fundo,fundo_cnpj,adm,adm_cnpj,pl,hora_recebimento = parse_html_tudo_int(apelido_fundo,anomes,candidatos=True)

Wall time: 777 ms


In [13]:
carteira_pandas,alfabetico = separa_blocos(carteira_pandas)

grupo 3 todos os ativos1 são nulos


In [14]:
set(carteira_pandas[carteira_pandas.grupo == 0].ativo1)

{'Cotas de Fundos',
 'Disponibilidades',
 'Investimento no Exterior',
 'Valores a pagar',
 'Valores a receber'}

In [15]:
def explodir(carteira_pandas,alfabetico):
    teste = carteira_pandas[carteira_pandas.grupo!=0]
    assert len(teste[teste.candidatos.isnull()]) ==0
    teste = pd.DataFrame(columns=['index','aserestudado','ativo1','ativo2','ativo3','ativo4','classificacao','emp_ligada','vendas_quant','vendas_montante','aquis_quant','aquis_montante','pf_quant','pf_valor_custo','pf_valor_mercado','porcent_partrimonio_liquido','fechado','candidatos','cand_pf_quant', 'grupo'],data=[(index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,c,q,grupo) if pd.isnull(ativo3) else (index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,ativo3,pf_quant,grupo) for (index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo) in teste.reset_index().values  for (c,q) in zip(candidatos,cand_pf_quant) ]).set_index('index')
    teste = teste.drop_duplicates().sort_values(by='aserestudado').reset_index()
    

    teste = filtro_alfabetico(teste,alfabetico)
    teste.pf_quant = teste.cand_pf_quant
    
    
    teste.pf_quant = teste.pf_quant.apply(abs)*teste.ativo1.apply(lambda x: -1 if 'VALORES A PAGAR' in x.upper() or 'OBRIGAÇÕES' in x.upper() else 1)
    teste.pf_valor_mercado = teste.pf_valor_mercado.apply(abs)*teste.ativo1.apply(lambda x: -1 if 'VALORES A PAGAR' in x.upper() or 'OBRIGAÇÕES' in x.upper() else 1)
    
    
    
    return teste

In [16]:
explodido = explodir(carteira_pandas,alfabetico)

In [17]:
def completa(ativo,linhas,teste):
    for i in linhas:
        teste.loc[int(i),'ativo3'] = ativo
    return teste

In [18]:
lista_acoes = ['Obrigações por ações e outros TVM recebidos em empréstimo',
                 'Ações e outros TVM cedidos em empréstimo',
                 'Ações',
                 'Certificado ou recibo de depósito de valores mobiliários', 
                 'Valores a pagar']

aux.pf_quant = aux.pf_quant.apply(abs)*aux.ativo1.apply(lambda x: -1 if 'VALORES A PAGAR' in x.upper() or 'OBRIGAÇÕES' in x.upper() else 1)
aux.pf_valor_mercado = aux.pf_valor_mercado.apply(abs)*aux.ativo1.apply(lambda x: -1 if 'VALORES A PAGAR' in x.upper() or 'OBRIGAÇÕES' in x.upper() else 1)
aux =  aux[aux.ativo1.isin(lista_acoes)]


In [19]:
aux.loc[5,'ativo3'] = 'BPAC11'

In [20]:
MES0 = aux.copy()
MES1 = explodido.copy()
for ativo in sorted(list({x for x in aux.ativo3 if pd.notna(x)})):
    if ativo.upper() not in ('MRSA5B','MRSA3B','MOAR3','EUCA4','USIM6','TIET12','Valores a Pagar'.upper()):
        MES1 = teste_fraco(MES0,MES1,ativo,'202004')

-------------------ARZZ3-------------------
B3SA3 -154860000.0 
 Empty DataFrame
Columns: [ativo1, aserestudado, vendas_montante, aquis_montante, pf_valor_mercado, cand_pf_quant, MAX, MIN]
Index: [] 
 []
-------------------BPAC11-------------------
CNTO3 59130000.0 
      ativo1  aserestudado  vendas_montante  aquis_montante  pf_valor_mercado  \
1888  Ações      58334064              0.0     676454700.0       27370011300   
2342  Ações      58334070              0.0     145085500.0        6455216400   

      cand_pf_quant           MAX           MIN  
1888      8661396.0  8.466452e+08  8.305367e+08  
2342      2042790.0  2.000979e+08  1.966429e+08   
 []
CSAN3 78340000.0 
 Empty DataFrame
Columns: [ativo1, aserestudado, vendas_montante, aquis_montante, pf_valor_mercado, cand_pf_quant, MAX, MIN]
Index: [] 
 []
CYRE3 170000.0 
      ativo1  aserestudado  vendas_montante  aquis_montante  pf_valor_mercado  \
1011  Ações      58334052              0.0     252126300.0        9074138000   



In [21]:
CONFUSAO = MES1.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3>1]

,ativo3
aserestudado,
58334063,2


In [22]:
#papeis que ele Não achou
CONFUSAO = MES1.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3<1]

,ativo3
aserestudado,
58334046,0
58334053,0
58334057,0
58334058,0
58334066,0


In [23]:
#Antes de rodar o filtro_alfabetico preciso me certicicar que não há nenhuma linha com mais de um valor na coluna ativo3, ou seja nenhum aserestudado_com_varios_ativo3, ou seja, que  len(CONFUSAO[CONFUSAO.ativo3>1]) == 0
assert len(CONFUSAO[CONFUSAO.ativo3>1]) == 0
MES1 = filtro_alfabetico(MES1,alfabetico)

AssertionError: 

In [24]:

assert len(CONFUSAO[CONFUSAO.ativo3>1]) != 0
MES1 = filtro_alfabetico(MES1,alfabetico,aserestudado_com_varios_ativo3=58334063)

In [25]:
#Não achou
nao_achou = MES1[MES1.aserestudado.isin(CONFUSAO[CONFUSAO.ativo3<1].index)]

In [26]:
MES0[MES0.ativo1.str.contains('Certificado')]

,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo
42,57772133,Certificado ou recibo de depósito de valores m...,Cod. Ativo:,SULA11,Dt. Ini. Vigen.:05/10/2007,Para negociacao,Nao,NaN,NaN,NaN,NaN,1511076.0,NaN,5107436880,1.755,False,"[ADHM3, ALUP3, AZEV3, BBRK3, BDLL4, BEEF3, BID...","[35223703, 4664326, 14592677, 30401410, 728593...",1
41,57772145,Certificado ou recibo de depósito de valores m...,Cod. Ativo:,ENGI11,Dt. Ini. Vigen.:03/11/2009,Para negociacao,Nao,NaN,NaN,NaN,NaN,2296011.0,NaN,8678921580,2.982,False,"[ADHM3, AMAR3, ATMP3, AZEV3, AZEV4, BBRK3, BEE...","[59854632, 18867221, 16072077, 24796919, 39811...",1


In [27]:
MES1[MES1.aserestudado==58334046][MES1.candidatos=='SULA11']

,index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo
665,32,58334046,Ações e outros TVM cedidos em empréstimo,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,258124.0,NaN,1155104900,0.0,True,SULA11,258124.0,2


In [28]:
MES1 = completa('SULA11',{665,2015},MES1)
MES1 = filtro_unico_index(MES1)
MES1

Empty DataFrame
Columns: [ativo3]
Index: []
              ativo3
aserestudado        
58334053           0
58334057           0
58334058           0


,index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo
0,35,58334034,Ações e outros TVM cedidos em empréstimo,NaN,ARZZ3,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,113579.0,NaN,482710700,0.0,True,ARZZ3,113579.0,2
28,35,58334034,Ações e outros TVM cedidos em empréstimo,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,3602319.0,NaN,482710700,0.0,True,ATOM3,3602319.0,2
29,35,58334034,Ações e outros TVM cedidos em empréstimo,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,207172.0,NaN,482710700,0.0,True,BGIP4,207172.0,2
30,35,58334034,Ações e outros TVM cedidos em empréstimo,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,2806458.0,NaN,482710700,0.0,True,BOBR4,2806458.0,2
31,35,58334034,Ações e outros TVM cedidos em empréstimo,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,161658.0,NaN,482710700,0.0,True,BRAP4,161658.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2462,7,58334079,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,269829400.0,23884336.0,NaN,12658698000,3.0,True,USIM3,23884336.0,3
2461,7,58334079,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,269829400.0,83280908.0,NaN,12658698000,3.0,True,VIVR3,83280908.0,3
2460,7,58334079,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,269829400.0,1346670.0,NaN,12658698000,3.0,True,VVPR11,1346670.0,3
2486,7,58334079,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,269829400.0,4366574.0,NaN,12658698000,3.0,True,WSON33,4366574.0,3


In [29]:
MES1[MES1.aserestudado==58334053]

,index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo
1106,37,58334053,Certificado ou recibo de depósito de valores m...,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,2394690.0,NaN,10464795300,3.0,True,ENGI11,2394690.0,1
1105,37,58334053,Certificado ou recibo de depósito de valores m...,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,34423669.0,NaN,10464795300,3.0,True,EQPA3,34423669.0,1
1104,37,58334053,Certificado ou recibo de depósito de valores m...,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,17470443.0,NaN,10464795300,3.0,True,EQPA7,17470443.0,1


In [30]:
MES1 = completa('ENGI11',{1106},MES1)
MES1 = filtro_unico_index(MES1)
MES1

Empty DataFrame
Columns: [ativo3]
Index: []
              ativo3
aserestudado        
58334057           0
58334058           0


,index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo
0,35,58334034,Ações e outros TVM cedidos em empréstimo,NaN,ARZZ3,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,113579.0,NaN,482710700,0.0,True,ARZZ3,113579.0,2
28,35,58334034,Ações e outros TVM cedidos em empréstimo,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,3602319.0,NaN,482710700,0.0,True,ATOM3,3602319.0,2
29,35,58334034,Ações e outros TVM cedidos em empréstimo,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,207172.0,NaN,482710700,0.0,True,BGIP4,207172.0,2
30,35,58334034,Ações e outros TVM cedidos em empréstimo,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,2806458.0,NaN,482710700,0.0,True,BOBR4,2806458.0,2
31,35,58334034,Ações e outros TVM cedidos em empréstimo,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,161658.0,NaN,482710700,0.0,True,BRAP4,161658.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2462,7,58334079,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,269829400.0,23884336.0,NaN,12658698000,3.0,True,USIM3,23884336.0,3
2461,7,58334079,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,269829400.0,83280908.0,NaN,12658698000,3.0,True,VIVR3,83280908.0,3
2460,7,58334079,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,269829400.0,1346670.0,NaN,12658698000,3.0,True,VVPR11,1346670.0,3
2486,7,58334079,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,269829400.0,4366574.0,NaN,12658698000,3.0,True,WSON33,4366574.0,3


In [31]:
CONFUSAO = MES1.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3<1]

,ativo3
aserestudado,
58334057,0
58334058,0


In [32]:
list(CONFUSAO[CONFUSAO.ativo3<1].index)

[58334057, 58334058]

In [33]:
MES1 = filtro_alfabetico(MES1,alfabetico)
CONFUSAO = MES1.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3<1]

,ativo3
aserestudado,
58334057,0
58334058,0


In [34]:
MES1 = filtro_unico_index(MES1,aserestudado_sem_nenhum_ativo3=list(CONFUSAO[CONFUSAO.ativo3<1].index))
#MES1 = MES1.set_index('index')

TRABALHO CONCLUIDO


In [35]:
MES1[MES1.aserestudado==58334057]

,index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo
1379,22,58334057,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,1.158770e+09,103683.0,NaN,1213091100,0.0,True,GRLV11,103683.0,1
1378,22,58334057,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,1.158770e+09,4665735.0,NaN,1213091100,0.0,True,GSFI11,4665735.0,1
1346,22,58334057,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,1.158770e+09,951444.0,NaN,1213091100,0.0,True,GUAR3,951444.0,1
1376,22,58334057,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,1.158770e+09,6384690.0,NaN,1213091100,0.0,True,HAGA4,6384690.0,1
1375,22,58334057,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,1.158770e+09,236470.0,NaN,1213091100,0.0,True,HALI34,236470.0,1
1374,22,58334057,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,1.158770e+09,4720199.0,NaN,1213091100,0.0,True,IDVL4,4720199.0,1
1373,22,58334057,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,1.158770e+09,6220980.0,NaN,1213091100,0.0,True,IGBR3,6220980.0,1
1372,22,58334057,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,1.158770e+09,1840806.0,NaN,1213091100,0.0,True,INEP4,1840806.0,1
1377,22,58334057,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,1.158770e+09,1347879.0,NaN,1213091100,0.0,True,ITSA4,1347879.0,1
1370,22,58334057,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,1.158770e+09,9862530.0,NaN,1213091100,0.0,True,JBDU4,9862530.0,1


In [36]:
MES1[MES1.aserestudado==58334058]

,index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo
1443,18,58334058,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,5.549048e+09,11352140.0,NaN,5517140000,1.0,True,GOAU3,11352140.0,1
1442,18,58334058,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,5.549048e+09,1379285.0,NaN,5517140000,1.0,True,GSHP3,1379285.0,1
1445,18,58334058,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,5.549048e+09,29037579.0,NaN,5517140000,1.0,True,HAGA4,29037579.0,1
1441,18,58334058,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,5.549048e+09,27178030.0,NaN,5517140000,1.0,True,HRDF11,27178030.0,1
1440,18,58334058,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,5.549048e+09,21467471.0,NaN,5517140000,1.0,True,IDVL4,21467471.0,1
1438,18,58334058,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,5.549048e+09,28293026.0,NaN,5517140000,1.0,True,IGBR3,28293026.0,1
1419,18,58334058,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,5.549048e+09,8371988.0,NaN,5517140000,1.0,True,INEP4,8371988.0,1
1433,18,58334058,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,5.549048e+09,5185282.0,NaN,5517140000,1.0,True,ITSA3,5185282.0,1
1417,18,58334058,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,5.549048e+09,44854797.0,NaN,5517140000,1.0,True,JBDU4,44854797.0,1
1402,18,58334058,Ações,NaN,NaN,NaN,Para negociacao,Nao,NaN,0.0,NaN,5.549048e+09,39979276.0,NaN,5517140000,1.0,True,JCPC34,39979276.0,1


In [37]:
MES1 = completa('LAME3',{1314},MES1)
MES1 = completa('LAME4',{1392},MES1)
MES1 = filtro_unico_index(MES1)
MES1

TRABALHO CONCLUIDO


,index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo
0,35,58334034,Ações e outros TVM cedidos em empréstimo,NaN,ARZZ3,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,113579.0,NaN,482710700,0.000,True,ARZZ3,113579.0,2
103,28,58334035,Ações e outros TVM cedidos em empréstimo,NaN,ENEV3,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,565105.0,NaN,2033247700,0.000,True,ENEV3,565105.0,2
182,23,58334036,Ações e outros TVM cedidos em empréstimo,NaN,ENGI11,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,956813.0,NaN,4181272800,1.000,True,ENGI11,956813.0,2
185,2,58334037,Ações e outros TVM cedidos em empréstimo,Cod. Ativo:,EQTL3,Dt. Ini. Vigen.:03/04/2006,Para negociacao,Nao,NaN,NaN,NaN,NaN,373863.0,NaN,685290879,0.201,False,EQTL3,373863.0,2
243,30,58334038,Ações e outros TVM cedidos em empréstimo,NaN,EZTC3,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,436519.0,NaN,1318723800,0.000,True,EZTC3,436519.0,2
312,33,58334039,Ações e outros TVM cedidos em empréstimo,NaN,GNDI3,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,184105.0,NaN,1009079500,0.000,True,GNDI3,184105.0,2
330,29,58334040,Ações e outros TVM cedidos em empréstimo,NaN,LREN3,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,384993.0,NaN,1477988100,0.000,True,LREN3,384993.0,2
412,31,58334041,Ações e outros TVM cedidos em empréstimo,NaN,MGLU3,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,255861.0,NaN,1271629100,0.000,True,MGLU3,255861.0,2
474,27,58334042,Ações e outros TVM cedidos em empréstimo,NaN,PSSA3,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,462771.0,NaN,2095889800,0.000,True,PSSA3,462771.0,2
551,36,58334043,Ações e outros TVM cedidos em empréstimo,NaN,RADL3,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,40060.0,NaN,419748600,0.000,True,RADL3,40060.0,2


In [38]:
MES1.sort_values(['aserestudado','candidatos']).to_csv('excel_fundos\\'+str(anomes)+'_2.xls',sep='\t',decimal= '.')

In [39]:
#aqui tenho q ver se as linhas com short estao realmente negativas
aux = MES1.copy()

In [40]:
anomes = 202005
apelido_fundo = 'CONST'.upper()


%time carteira_pandas,fundo,fundo_cnpj,adm,adm_cnpj,pl,hora_recebimento = parse_html_tudo_int(apelido_fundo,anomes,candidatos=True)

Wall time: 785 ms


In [41]:
carteira_pandas,alfabetico = separa_blocos(carteira_pandas)

ISSO ESTA ERRADO, QUANDO É NÃO ALFABETICO (olhar squadra 202004 o aserestudado 58331405 q vc vai entender)
Tento by passar usando a localização de algum Investimento no Exterior
grupo 3 todos os ativos1 são nulos


In [42]:
explodido = explodir(carteira_pandas,alfabetico)

In [43]:
MES1 = aux.copy()
MES2 = explodido.copy()
for ativo in sorted(list({x for x in aux.ativo3 if pd.notna(x)})):
    if ativo.upper() not in ('MRSA5B','MRSA3B','MOAR3','EUCA4','USIM6','TIET12','Valores a Pagar'.upper()):
        MES2 = teste_fraco(MES1,MES2,ativo,'202005')

-------------------ARZZ3-------------------
B3SA3 -2320000.0 
 Empty DataFrame
Columns: [ativo1, aserestudado, vendas_montante, aquis_montante, pf_valor_mercado, cand_pf_quant, MAX, MIN]
Index: [] 
 []
-------------------BPAC11-------------------
-------------------ENEV3-------------------
ENGI11 335150300.0 
                                                  ativo1  aserestudado  \
1314  Certificado ou recibo de depósito de valores m...      58845430   
2367           Ações e outros TVM cedidos em empréstimo      58845447   

      vendas_montante  aquis_montante  pf_valor_mercado  cand_pf_quant  \
1314              0.0             0.0       12385264000      2607424.0   
2367              0.0             0.0        4016975200       845679.0   

               MAX           MIN  
1314  2.607424e+08  2.607424e+08  
2367  8.456790e+07  8.456790e+07   
 []
EQTL3 406410000.0 
                                         ativo1  aserestudado  vendas_montante  \
743                               

In [44]:
CONFUSAO = MES2.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3>1]

,ativo3
aserestudado,


In [45]:
CONFUSAO = MES2.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3<1]

,ativo3
aserestudado,
58845414,0
58845417,0
58845420,0
58845426,0
58845428,0
58845429,0
58845430,0
58845434,0
58845437,0


In [46]:
#Antes de rodar o filtro_alfabetico preciso me certicicar que não há nenhuma linha com mais de um valor na coluna ativo3, ou seja nenhum aserestudado_com_varios_ativo3, ou seja, que  len(CONFUSAO[CONFUSAO.ativo3>1]) == 0
assert len(CONFUSAO[CONFUSAO.ativo3>1]) == 0
MES2 = filtro_alfabetico(MES2,alfabetico)

In [47]:
#Não achou
nao_achou = MES2[MES2.aserestudado.isin(CONFUSAO[CONFUSAO.ativo3<1].index)]

In [48]:
MES1[MES1.ativo1.str.contains('Certificado')]

,index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo
1106,37,58334053,Certificado ou recibo de depósito de valores m...,NaN,ENGI11,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,2394690.0,NaN,10464795300,3.0,True,ENGI11,2394690.0,1
2015,38,58334066,Certificado ou recibo de depósito de valores m...,NaN,SULA11,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,1215776.0,NaN,5440597600,1.0,True,SULA11,1215776.0,1


''

In [73]:
MES2 = completa('ENGI11',{1314,2367},MES2)
MES2 = completa('KLBN11',{1007},MES2)
MES2 = completa('TOTS3',{183,1720},MES2) #Teve aumento de capital
MES2 = completa('RENT3',{377,1755},MES2) 
MES2 = completa('EQTL3',{2297,1251},MES2) 
MES2 = completa('EZTC3',{2215,1204},MES2) 
MES2 = completa('PSSA3',{506,1883},MES2) 
MES2 = completa('ALPK3',{1507},MES2) 
MES2 = completa('ALUP11',{2659},MES2) 

In [74]:
CONFUSAO = MES2.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3<1]

,ativo3
aserestudado,


In [75]:
MES2 = filtro_unico_index(MES2,aserestudado_sem_nenhum_ativo3=list(CONFUSAO[CONFUSAO.ativo3<1].index))
#MES2.set_index('index')

TRABALHO CONCLUIDO


In [52]:
MES2[MES2.aserestudado==58845455]

,index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo
2659,37,58845455,Brazilian Depository Receipt - BDR,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,34782.0,NaN,86502800,0.0,True,ALUP11,34782.0,3
2660,37,58845455,Brazilian Depository Receipt - BDR,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,7683.0,NaN,86502800,0.0,True,ALZR11,7683.0,3
2661,37,58845455,Brazilian Depository Receipt - BDR,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,39445.0,NaN,86502800,0.0,True,ANIM3,39445.0,3
2662,37,58845455,Brazilian Depository Receipt - BDR,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,20547.0,NaN,86502800,0.0,True,ARZZ3,20547.0,3
2663,37,58845455,Brazilian Depository Receipt - BDR,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,537285.0,NaN,86502800,0.0,True,ATOM3,537285.0,3
2664,37,58845455,Brazilian Depository Receipt - BDR,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,221235.0,NaN,86502800,0.0,True,AZEV3,221235.0,3
2665,37,58845455,Brazilian Depository Receipt - BDR,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,247151.0,NaN,86502800,0.0,True,AZEV4,247151.0,3


In [53]:
#aqui tenho q ver se as linhas com short estao realmente negativas
aux = MES2.copy()

In [54]:
anomes = 202006
apelido_fundo = 'CONST'.upper()


%time carteira_pandas,fundo,fundo_cnpj,adm,adm_cnpj,pl,hora_recebimento = parse_html_tudo_int(apelido_fundo,anomes,candidatos=True)

Wall time: 847 ms


In [55]:
carteira_pandas,alfabetico = separa_blocos(carteira_pandas)

grupo 3 todos os ativos1 são nulos
grupo 4 todos os ativos1 são nulos


In [56]:
alfabetico

True

In [57]:
set(carteira_pandas[carteira_pandas.grupo == 0].ativo1)

{'Cotas de Fundos',
 'Disponibilidades',
 'Investimento no Exterior',
 'Outros valores mobiliários registrados na CVM objeto de oferta pública',
 'Valores a pagar',
 'Valores a receber'}

In [58]:
explodido = explodir(carteira_pandas,alfabetico)

In [59]:
MES2 = aux.copy()
MES3 = explodido.copy()
for ativo in sorted(list({x for x in aux.ativo3 if pd.notna(x)})):
    if ativo.upper() not in ('MRSA5B','MRSA3B','MOAR3','EUCA4','USIM6','TIET12','Valores a Pagar'.upper()):
        MES3 = teste_fraco(MES2,MES3,ativo,'202006')

-------------------ALPK3-------------------
ARZZ3 92080000.0 
                                        ativo1  aserestudado  vendas_montante  \
134  Ações e outros TVM cedidos em empréstimo      59602657              0.0   

     aquis_montante  pf_valor_mercado  cand_pf_quant           MAX  \
134             0.0        2515487600       525154.0  5.251540e+07   

              MIN  
134  5.251540e+07   
 []
B3SA3 117750000.0 
 Empty DataFrame
Columns: [ativo1, aserestudado, vendas_montante, aquis_montante, pf_valor_mercado, cand_pf_quant, MAX, MIN]
Index: [] 
 []
-------------------BPAC11-------------------
-------------------ENEV3-------------------
ENGI11 345310300.0 
                                                  ativo1  aserestudado  \
85             Ações e outros TVM cedidos em empréstimo      59602656   
1246  Certificado ou recibo de depósito de valores m...      59602675   

      vendas_montante  aquis_montante  pf_valor_mercado  cand_pf_quant  \
85                0.0      

In [60]:
CONFUSAO = MES3.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3>1]

,ativo3
aserestudado,


In [69]:
#papeis que ele Não achou
CONFUSAO = MES3.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3<1]

,ativo3
aserestudado,


In [64]:
MES3 = completa('ENGI11',{85,1246},MES3)
MES3 = completa('KLBN11',{1412},MES3)
MES3 = completa('LAME4',{336,1590},MES3)


In [70]:
#Antes de rodar o filtro_alfabetico preciso me certicicar que não há nenhuma linha com mais de um valor na coluna ativo3, ou seja nenhum aserestudado_com_varios_ativo3, ou seja, que  len(CONFUSAO[CONFUSAO.ativo3>1]) == 0
assert len(CONFUSAO[CONFUSAO.ativo3>1]) == 0
MES3 = filtro_alfabetico(MES3,alfabetico)

In [71]:
MES3 = filtro_unico_index(MES3,aserestudado_sem_nenhum_ativo3=list(CONFUSAO[CONFUSAO.ativo3<1].index))
#MES2.set_index('index')

TRABALHO CONCLUIDO


In [65]:
MES0.to_csv('excel_fundos\\'+apelido_fundo+'\\'+str(202003)+'.xls',sep='\t',decimal= '.')

In [66]:
MES1.to_csv('excel_fundos\\'+apelido_fundo+'\\'+str(202004)+'.xls',sep='\t',decimal= '.')

In [76]:
MES2.to_csv('excel_fundos\\'+apelido_fundo+'\\'+str(202005)+'.xls',sep='\t',decimal= '.')

In [72]:
MES3.to_csv('excel_fundos\\'+apelido_fundo+'\\'+str(202006)+'.xls',sep='\t',decimal= '.')